In [3]:
#import osmnx as ox
#import networkx as nx
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [17]:
busdf=pd.read_csv('buses.csv')
busdf.head()

,timestamp,ID,type,is_driving,is_parked,n_passengers,total_km
0,1,bus_0,BUS_L,1,0,0,2.600000e+00
1,2,bus_0,BUS_L,1,0,0,7.800000e+00
2,3,bus_0,BUS_L,1,0,0,1.000000e+01
3,4,bus_0,BUS_L,0,1,0,-1.073742e+09
4,4,bus_1,BUS_L,1,0,0,0.000000e+00


In [18]:
busdf.drop(columns=['type'],inplace=True)

In [14]:
bids=busdf.ID.unique()

array(['bus_0', 'bus_1', 'bus_2', 'bus_3', 'bus_4', 'bus_5', 'bus_6',
       'bus_7', 'bus_8', 'bus_9', 'bus_10', 'bus_11', 'bus_12', 'bus_13',
       'bus_14', 'bus_15', 'bus_16', 'bus_17', 'bus_18', 'bus_19',
       'bus_20', 'bus_21', 'bus_22', 'bus_23', 'bus_24', 'bus_25',
       'bus_26', 'bus_27', 'bus_28', 'bus_29', 'bus_30', 'bus_31',
       'bus_32', 'bus_33', 'bus_34', 'bus_35', 'bus_36', 'bus_37',
       'bus_38', 'bus_39', 'bus_40', 'bus_41', 'bus_42', 'bus_43',
       'bus_44', 'bus_45', 'bus_46', 'bus_47', 'bus_48', 'bus_49'],
      dtype=object)

In [92]:
mlist=[]
for b in bids:
    bdf=busdf.where(busdf.ID==b).dropna()
    mlist.append(hour_bin(bdf))
masterdf=pd.concat(mlist)

In [89]:
def hour_bin(dfin):
    #hdf=pd.DataFrame(columns=['hour','bus_id','is_driving','is_parked','n_passengers','total_km'])
    dflist=[]
    for i in range(0,25):
        tstart=i*3600
        tend=(i+1)*3600
        trange=dfin.query('timestamp >= @tstart & timestamp < @tend').dropna() #dataframe within hour
        if len(trange) >0:
            hour=i
            bus_id=list(dfin.ID.unique())[0]
            is_driving=trange.is_driving.mean()
            is_parked=trange.is_parked.mean()
            n_pass=trange.n_passengers.mean()
            kms=[t for t in trange.total_km if t >0.]
            kms.sort()
            #print(kms[0],kms[-1])
            total_km=(kms[-1]-kms[0])/1000.#np.sum([t for t in trange.total_km if t >0.]) #trange.total_km.sum() #need to ignore negatives
            nsteps=len(trange)
            #print(hour,bus_id,is_driving,is_parked,n_pass,total_km)
            tdf=pd.DataFrame({'hour':hour,'bus_id':bus_id,'is_driving':is_driving,'is_parked':is_parked,'n_passengers':n_pass,'total_km':total_km,'nsteps':nsteps},index=[i])
            dflist.append(tdf)
    if len(dflist) !=0:
        hdf=pd.concat(dflist)
    return hdf

In [102]:
masterdf['occupancy']=(masterdf.n_passengers)*100.

In [95]:
masterdf.reset_index(drop=True,inplace=True)

In [104]:
masterdf.to_json('buses.json')

In [103]:
masterdf.head()

,hour,bus_id,is_driving,is_parked,n_passengers,total_km,nsteps,occupancy
0,0,bus_0,0.205057,0.793554,0.022506,7.826068,3599,2.250625
1,1,bus_0,0.314794,0.684093,0.225806,12.794301,3596,22.580645
2,2,bus_0,0.356745,0.641864,0.223922,6.135026,1438,22.392211
3,0,bus_1,0.212736,0.785873,0.063682,7.470425,3596,6.368187
4,1,bus_1,0.372636,0.626251,0.212180,15.562430,3596,21.218020


In [113]:
import xml.etree.ElementTree as etree

tree = etree.parse("/Users/wheatley/Documents/Tech/out.xml")
root = tree.getroot()


In [182]:
#columns = ["name", "email", "age"]
tss=[]
pids=['person_'+str(i) for i in range(0,209)]#there are 208 people
pids.append('timestamp')
#dataframe = pd.DataFrame(columns = pids)
dfl=[]
for i,node in enumerate(root): 
    ts=float(node.attrib['time'])
    #dataframe.timestamp[i]=ts
    #print(ts)
    children = node.getchildren()
    pelem=[c for c in children if 'person' in c.tag]
    if pelem != []:
        #pedestrian waiting
        id_waiting=[p.attrib['id'] for p in pelem if p.attrib['speed']=='0.00']
        id_moving=[p.attrib['id'] for p in pelem if p.attrib['speed']!='0.00']
    tdict={'timestamp':ts}
    for p in pids[:-1]:
        if p not in id_waiting and p not in id_moving:
            tdict[p]=-1
        elif p in id_waiting:
            tdict[p]=0
        elif p in id_moving:
            tdict[p]=1
    ndf=pd.DataFrame(tdict,index=[i])
    dfl.append(ndf)
    #print(node.tag, node.attrib, node.text )
    #mail = node.find("email").text if node is not None else None
    #age = node.find("age").text if node is not None else None
#datatframe = datatframe.append(pd.Series([name, mail, age], index = columns), ignore_index = True)

/Users/wheatley/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:11: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [222]:
dft=pd.concat(dfl)

In [224]:
#p0=dft.person_0
#get changes in status
#try:
dl2=[]
for i,p in enumerate(pids[:-1]):
    tspawn=dft.where(dft[p] ==0).dropna()
    try:
        tstamp=list(tspawn.index)[0] #change to hour
        shour=tstamp #% 3600
    except IndexError: #nevers spawned
        shour=-1
        sb=-1
        sdr=-1
        ndf=pd.DataFrame({'id':p,'hour_spawn':shour,'hour_board':sb,'hour_drop':sdr},index=[i])
        dl2.append(ndf)
        pass
    try:
        #now find when they were dropped off
        tboard=dft.where(dft[p][tstamp:] ==1).dropna()
        tb=list(tboard.index)[0] 
        tdrop=dft.where(dft[p][tb:] ==0).dropna()
        tdr=list(tdrop.index)[0] 
        sb=tb #% 3600
        sdr=tdr #% 3600
    except IndexError: #theny never got on a bus
        sb=-1
        sdr=-1
        
    ndf=pd.DataFrame({'id':p,'hour_spawn':shour,'hour_board':sb,'hour_drop':sdr},index=[i])
    dl2.append(ndf)

In [225]:
df2=pd.concat(dl2)
df2.head()

,id,hour_spawn,hour_board,hour_drop
0,person_0,1177,1178,1196
1,person_1,242,515,552
2,person_2,1679,1680,1722
3,person_3,1469,1470,1543
4,person_4,628,629,652


In [236]:
df2['hd']=df2.h_drop.astype(int)
df2['hs']=df2.h_spawn.astype(int)
df2['hb']=df2.h_board.astype(int)

In [239]:
df2.drop(columns=['h_drop','h_spawn','h_board'],inplace=True)

In [241]:
df2.to_json('passengers.json')

In [240]:
df2.tail()

,id,hour_spawn,hour_board,hour_drop,hd,hs,hb
220,person_204,-1,-1,-1,0,0,0
221,person_205,0,8300,8312,2,0,2
222,person_206,0,8343,8394,2,0,2
223,person_207,0,8432,8553,2,0,2
224,person_208,8149,8150,8244,2,2,2


In [328]:
ss=pd.read_csv('simulation_scores.csv')
ss.head()

,Buses,Time,Score,passengers_not_arrived,total_distance,total_wait
0,12,71,75,63,767.671,7.15
1,16,74,64,48,910.296,7.85
2,20,77,65,45,992.628,8.02
3,22,78,61,39,1030.260,8.45
4,28,85,55,27,1123.856,8.80


In [334]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=ss.Buses,y=ss[' Score'],name='Score'))
fig.add_trace(go.Scatter(x=ss.Buses,y=ss[' total_wait'],name='Total wait time (h)'))
fig.add_trace(go.Scatter(x=ss.Buses,y=ss[' passengers_not_arrived'],name='Passengers not delivered'))
fig.add_trace(go.Scatter(x=ss.Buses,y=ss[' total_distance']/10.,name='Distance Driven (10 km)'))
fig.update_layout(title='Optimization Progress',xaxis_title='Number of Buses')